In [1]:
import pandas as pd
import numpy as np
import os

data_path = f'{os.path.dirname(os.path.dirname(os.getcwd()))}/dataset/'
data = pd.read_parquet(f'{data_path}/dataset_clean.parquet')

#splitting dataset into train set (80%), validation set (10%) and test set (10%)
data_train, data_val, data_test = np.split(data.sample(frac=1, random_state=42), [int(.8*len(data)), int(.9*len(data))])

print(len(data_train),len(data_val), len(data_test))

557638 69705 69705
